In [3]:
import os
os.environ['PYTHONHASHSEED'] = '0'
import nibabel as nib
import numpy as np
import yaml

from hashlib import md5

In [4]:
def trk2seeds(config):
    np.random.seed(42)
    
    subject_dir = os.path.join("subjects", config["subject"])
    
    trk_path = os.path.join(subject_dir, config["trk_path"])
    
    hasher = md5()
    for v in config.values():
        hasher.update(str(v).encode())
    
    save_dir = os.path.join(subject_dir, "seeds", hasher.hexdigest())
    
    if os.path.exists(save_dir):
        print("Seeds with this config have been created already:\n{}".format(save_dir))
        return
    
    tracts = nib.streamlines.load(trk_path).tractogram.streamlines # voxel-center coords
    
    seeds = []
    for fiber in tracts:
        seeds.append(fiber[0])
        seeds.append(fiber[-1])
    seeds = np.array(seeds)
    
    rand_shifts = np.random.uniform(-config["rand_radius"], config["rand_radius"],
                                    size=(len(seeds), config["rand_samples"], 3)
    )
    
    rand_seeds = rand_shifts + seeds.reshape(-1, 1, 3)
    rand_seeds = rand_seeds.reshape(-1, 3)
    
    all_seeds = np.vstack([seeds, rand_seeds])
    
    all_seeds = np.hstack([all_seeds, np.ones((len(all_seeds), 1))]) # Additional dimension for 4x4 affine
    
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, "seeds.npy")
    print("Saving {}".format(save_path))
    np.save(save_path, all_seeds) # We don't need to shift by +0.5 vox, because we load again with numpy
    
    config["n_seeds"] = len(all_seeds)
    config_path = os.path.join(save_dir, "config" + ".yml")
    print("Saving {}".format(config_path))
    with open(config_path, "w") as file:
            yaml.dump(config, file, default_flow_style=False)
    
    return all_seeds

In [7]:
config=dict(
    subject="ismrm_rpe",
    trk_path="tracts/merged.trk",
    rand_radius=1.0,
    rand_samples=1
)

In [8]:
seeds = trk2seeds(config)

Saving subjects/ismrm_rpe/seeds/6c78e951274fe8b719147ec639daddbe/seeds.npy
Saving subjects/ismrm_rpe/seeds/6c78e951274fe8b719147ec639daddbe/config.yml
